----

## Initial Setup

In [6]:
import dotenv
import os
import pandas as pd
import sys

from pathlib import Path

In [7]:
# Load and set environment

dotenv.load_dotenv()
os.environ['USER_AGENT'] = 'myagent'
PROJECT_HOME = Path(os.environ.get('PROJECT_HOME', Path.cwd() / '..')).resolve()
sys.path.append(str(PROJECT_HOME))

In [8]:
from app.databases.vector.milvus import Milvus
from app.databases.vector import VectorDB

vector_db = Milvus(
    # auto_id=True,
    # drop_old=True,  # Drop existing values inside the collection
)

----

## Getting a list of current documents

In [26]:
def display_docs():
    # We can't do an empty search, so we'll just search on ".".
    all_docs = vector_db.similarity_search(".", k=100)  # k is the number of docs to retrieve
    
    # Display the documents
    for doc in all_docs:
        print(f"ID: {doc.metadata.get('source_id', 'N/A')} | Text: {doc.page_content[:50]}...")

In [104]:
display_docs()

ID: 3 | Text: testing anohter text...
ID: 4 | Text: I want to try my hand at RAG...
ID: 1001 | Text: The recommended headphones to use while listening ...


----

## Adding text to the vector db

In [88]:
from datetime import datetime

class StoreTextWrap:
    def __init__(self):
        self.cur_source_id = 1
    
    def store(self, text, source_name="chat", source_id=None, modified_at=None):
        
        if not source_id:
            source_id = str(self.cur_source_id)
            self.cur_source_id += 1

        modified_at = modified_at or datetime.now()
        
        return vector_db.split_and_store_text(text,
            metadata={
                'source_name':source_name,
                'source_id':source_id,
                'modified_at':modified_at.isoformat(),
            })
store_text = StoreTextWrap().store
# Use by just calling store_text().

In [102]:
await store_text("I want to try my hand at RAG")

[453544779837774764]

----

## Deleting text from the db

In [97]:
def delete_by_id(aid):
    return vector_db.delete_embeddings(aid)


In [100]:

await delete_by_id(2)

{'insert_count': 0,
 'delete_count': 2,
 'upsert_count': 0,
 'timestamp': 0.0,
 'success_count': 0,
 'error_count': 0,
 'error_index': '[]'}